In [1]:
import phoebe
from ipywidgets import *
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
lc = np.loadtxt('../../../thuban/data/adra_sap_spline_corrected.txt', usecols=(0,1,2))
rv = np.loadtxt('../../../thuban/data/bischoff_rv_fixed_time.txt')

In [3]:
b = phoebe.default_binary()

period = 51.41923
rsum = 0.07213
rratio = 0.62877
incl = 86.17310
sbratio = 0.76078
eccen = 0.41346
omega = 0.27860
sma = 43.719243
tratio=0.8
esinw = eccen*np.sin(omega)
ecosw = eccen*np.cos(omega)
t0_supconj = 1696.0197281834112

b['t0_supconj'] = t0_supconj
b['period@binary'] = period
b['incl@binary'] = incl
b['sma@binary'] = sma
b.flip_constraint('esinw', solve_for='ecc')
b.flip_constraint('ecosw', solve_for='per0')
b['esinw'] = eccen*np.sin(omega)
b['ecosw'] = eccen*np.cos(omega)
b['teff@primary'] = 10100
b['teff@secondary'] = tratio*b['value@teff@primary']
b['requiv@primary'] = b['value@sma@binary']*rsum/(1.+rratio)
b['requiv@secondary'] = b['value@sma@binary']*rsum*rratio/(1.+rratio)

phases = np.linspace(-0.5,0.5,101)
b.add_dataset('lc', times=lc[:,0], fluxes=lc[:,1], sigmas=lc[:,2], compute_phases=phases)
b.add_dataset('rv', component='primary', times=rv[:,0], rv=rv[:,1], sigmas=rv[:,2])
b.run_compute(irrad_method='none', ntriangles=400)

# b['pblum_mode'] = 'dataset-scaled'
# b.set_value_all('atm', 'blackbody')
# b.set_value_all('ld_mode', 'manual')
# b.set_value_all('ld_mode_bol', 'manual')
# b.set_value_all('ld_coeffs_bol', [0.5,0.5])

<ParameterSet: 6 parameters | contexts: model, figure>

In [4]:
def make_slider_text(value, minv, maxv, step, description):
    slider = FloatSlider(
        value=value,
        min=minv,
        max=maxv,
        step=step,
        description=description,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=False,
        readout_format='.5f',
        layout=Layout(width='200px')
    )

    text = FloatText(
        value=value,
        #description=description,
        disabled=False,
        layout=Layout(width='85px'),
    )
    return slider, text


def link(slider, text):
    widgetLink = widgets.jslink((slider, 'value'), (text, 'value'))
    
    
def update_vals_in_bundle(period=period, rsum=rsum, rratio=rratio, incl=incl,
                        esinw=eccen*np.sin(omega), ecosw=eccen*np.cos(omega), sma=sma, q=1.0, tratio=tratio,
                        pblum=12, lc=True, rv=False):
    [period, rsum, rratio, incl, esinw, ecosw, tratio, sma, q, pblum] = [s.value for s in sliders]
    
    b['period@binary'] = period
    b['incl@binary'] = incl
    b['sma@binary'] = sma
    b['esinw'] = esinw
    b['ecosw'] = ecosw
    b['teff@secondary'] = tratio*b['value@teff@primary']
    b['requiv@primary'] = b['value@sma@binary']*rsum/(1.+rratio)
    b['requiv@secondary'] = b['value@sma@binary']*rsum*rratio/(1.+rratio)
    b['q'] = q
    b['pblum@primary'] = pblum

    b.run_compute(irrad_method='none', ntriangles=500)

def on_lc_button_clicked(button):
    
    for s in sliders:
        s.observe(update_vals_in_bundle, 'value')
    
    with plot:
        clear_output()
        plt.figure(figsize=(10,5))
        plt.plot(b['value@times@lc@dataset'], b['fluxes@lc@model'].interp_value(times=lc[:,0]), '-', c='#008699')
        plt.scatter(b['value@times@lc@dataset'], b['value@fluxes@lc@dataset'], c='#3f3535')
        plt.xlabel('Time (JD)')
        plt.ylabel('Flux (norm)')
        plt.xlim([1730,1750])
        plt.show()

def on_rv_button_clicked(button):

    for s in sliders:
        s.observe(update_vals_in_bundle, 'value')
    with plot:
        clear_output()
        plt.figure(figsize=(10,5))
        plt.plot(b['value@times@rv@primary@model'], b['value@rvs@rv@primary@model'], '-', c='#008699')
        plt.scatter(b['value@times@rv@primary@dataset'], b['value@rvs@rv@primary@dataset'], c='#3f3535')
        plt.xlabel('Time (JD)')
        plt.ylabel('RVs (km/s)')
        plt.show()

In [5]:
# parameter sliders and textboxes
period_slider, period_text = make_slider_text(period, period-0.1, period+0.1, 0.0001, 'Period:')
link(period_slider, period_text)
rsum_slider, rsum_text = make_slider_text(rsum, rsum-0.05, rsum+0.05, 0.0001, 'r1+r2:')
link(rsum_slider, rsum_text)
rratio_slider, rratio_text = make_slider_text(rratio, rratio-0.1, rratio+0.1, 0.0001, 'r2/r1:')
link(rratio_slider, rratio_text)
incl_slider, incl_text = make_slider_text(incl, incl-5, incl+5, 0.1, 'incl:')
link(incl_slider, incl_text)
esinw_slider, esinw_text = make_slider_text(esinw, esinw-0.1, esinw+0.1, 0.0001, 'esinw:')
link(esinw_slider, esinw_text)
ecosw_slider, ecosw_text = make_slider_text(ecosw, ecosw-0.1, ecosw+0.1, 0.0001, 'ecosw:')
link(ecosw_slider, ecosw_text)
sma_slider, sma_text = make_slider_text(sma, sma-5, sma+5, 0.001, 'sma:')
link(sma_slider, sma_text)
q_slider, q_text = make_slider_text(1., 0.5, 1.5, 0.001, 'q:')
link(q_slider, q_text)
tratio_slider, tratio_text = make_slider_text(tratio, 0.2, 1.2, 0.001, 'T2/T1:')
link(tratio_slider, tratio_text)
pblum_slider, pblum_text = make_slider_text(12, 5, 20, 0.001, 'pblum:')
link(pblum_slider, pblum_text)

sliders = [period_slider, rsum_slider, rratio_slider, incl_slider, 
                 esinw_slider, ecosw_slider, tratio_slider, sma_slider, q_slider, pblum_slider]
textboxes = [period_text, rsum_text, rratio_text, incl_text, 
                 esinw_text, ecosw_text, tratio_text, sma_text, q_text, pblum_text]

# plot buttons
lc_button = Button(description="Plot LC")
rv_button = Button(description="Plot RV")
plot = Output()

lc_button.on_click(on_lc_button_clicked)
rv_button.on_click(on_rv_button_clicked)


# lc/rv range sliders
# xmin_lc, xmax_lc = b['value@times@lc@dataset'].min(), b['value@times@lc@dataset'].min()
# ymin_lc, ymax_lc = b['value@fluxes@lc@dataset'].min(), b['value@fluxes@lc@dataset'].min()

xlim = FloatRangeSlider(
    value=[5, 7.5],
    min=0,
    max=10.0,
    step=0.1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

ylim = FloatRangeSlider(
    value=[5, 7.5],
    min=0,
    max=10.0,
    step=0.1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

left_box = VBox(sliders)
middle_box = VBox(textboxes)
button_row = HBox([lc_button, rv_button])
right_box = VBox([button_row, plot])
    
# fig, ax = plt.subplots(nrows=1, ncols=2)
display(HBox([left_box, middle_box, right_box]))